# タスク 4: 会話型インターフェース - Anthropic Claude LLM とのチャット

このノートブックでは、Amazon Bedrock の Anthropic Claude Foundation Models (FM) を使用してチャットボットを構築します。

チャットボットや仮想アシスタントなどの会話型インターフェースは、顧客のユーザーエクスペリエンスを向上させることができます。チャットボットは、自然言語処理 (NLP) と機械学習アルゴリズムを使用して、ユーザーのクエリを理解して応答します。チャットボットは、カスタマーサービス、販売、e コマースなどのさまざまなアプリケーションで使用して、ユーザーに迅速かつ効率的に応答できます。ユーザーは、Web サイト、ソーシャル メディア プラットフォーム、メッセージング アプリなどのさまざまなチャネルを通じてチャットボットにアクセスできます。

- **チャットボット** (基本)、 FM モデルを使用したゼロショット チャットボット
- **プロンプトを使用したチャットボット**、テンプレート (LangChain) - プロンプトテンプレートで提供されるコンテキストを持つチャットボット
- **ペルソナを使用したチャットボット**、定義されたロールを持つチャットボット。つまり、キャリアコー​​チと人間のやり取り
- **コンテキスト認識型チャットボット**、埋め込みを生成することで外部ファイルを介してコンテキストを渡します。

## Amazon Bedrock でチャットボットを構築するための LangChain フレームワーク

チャットボットなどの会話型インターフェイスでは、短期的にも長期的にも、以前のやり取りを記憶することが非常に重要になります。

LangChain フレームワークは、2 つの形式でメモリ コンポーネントを提供します。まず、LangChain は以前のチャット メッセージを管理および操作するためのヘルパーユーティリティを提供します。これらはモジュールとして機能するように設計されています。次に、LangChain はこれらのユーティリティをチェーンに組み込む簡単な方法を提供し、さまざまなタイプの抽象化を簡単に定義して操作できるようにすることで、強力なチャットボットを簡単に構築できるようにします。

## コンテキストを使用したチャットボットの構築 - 主要な要素

コンテキスト認識型チャットボットを構築する最初のプロセスは、コンテキストの埋め込みを生成することです。通常、取り込みプロセスは埋め込みモデルを実行し、埋め込みを生成して、ベクターストアに保存します。このノートブックでは、Titan Embeddings モデルを使用します。2 番目のプロセスは、ユーザー リクエストのオーケストレーション、インタラクション、呼び出し、および結果の返却です。これには、ユーザー リクエストのオーケストレーション、情報収集に必要なモデル/コンポーネントとのインタラクション、応答を作成するためのチャットボットの呼び出し、およびチャットボットの応答をユーザーに返すことが含まれます。

## Task 4.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [41]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


### Resource Optimization Utilities

In this section, we create a decorator that optimizes API resource usage and improves request reliability. This decorator implements several best practices for working with cloud-based AI services

In [42]:
import time
import random
from functools import wraps

# Resource optimization decorator for API efficiency
def optimize_resource_usage(max_attempts=10, base_pause=10.0):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_attempts):
                try:
                    # Space out requests for optimal service performance
                    if attempt > 0:
                        time.sleep(10.0)
                    return func(*args, **kwargs)
                except Exception as e:
                    error_str = str(e)
                    # Check if this is a service capacity exception
                    if any(err in error_str for err in ["ThrottlingException", "TooManyRequests", 
                                                     "Too many tokens", "Rate exceeded"]):
                        if attempt < max_attempts - 1:
                            # Implement progressive backoff with jitter
                            jitter = random.random() * 0.5
                            wait_time = min(base_pause * (2 ** attempt) + jitter, 60)
                            
                            print(f"⏱️ Optimizing request timing. Pausing for {wait_time:.1f}s (attempt {attempt+1}/{max_attempts})")
                            time.sleep(wait_time)
                        else:
                            print("⚠️ Maximum attempts reached. Consider spacing out operations.")
                            raise
                    else:
                        raise
        return wrapper
    return decorator

### Understanding Resource Optimization

The code above implements an intelligent approach to API management that:

- **Spaces out requests** to maintain optimal throughput
- **Automatically retries** when temporary capacity limits are reached
- **Implements exponential backoff** by progressively increasing wait times
- **Adds randomized jitter** to prevent request clustering

These techniques are standard industry practices for working with cloud-based AI services and ensure your application remains responsive even under challenging conditions.

### Enhanced Bedrock Model Classes

Next, we create specialized versions of the Bedrock model classes that automatically incorporate the resource optimization techniques. These classes extend the standard LangChain implementations while adding reliability features that are essential for production applications.

By wrapping the critical API methods with our optimization decorator, we ensure consistent behavior when communicating with the Bedrock services.

In [43]:
from langchain_aws import ChatBedrock
from langchain_aws.embeddings import BedrockEmbeddings

# Enhanced ChatBedrock with resource optimization
class ResourceOptimizedChatBedrock(ChatBedrock):
    """Wrapper that optimizes resource usage and handles service capacity constraints."""
    
    @optimize_resource_usage(max_attempts=10)
    def _call_model(self, *args, **kwargs):
        return super()._call_model(*args, **kwargs)
    
    @optimize_resource_usage(max_attempts=10)
    def _generate(self, *args, **kwargs):
        return super()._generate(*args, **kwargs)
        
# Enhanced BedrockEmbeddings with resource optimization
class ResourceOptimizedEmbeddings(BedrockEmbeddings):
    """Wrapper for BedrockEmbeddings with intelligent resource management."""
    
    @optimize_resource_usage(max_attempts=10)
    def _embed_documents(self, texts):
        return super()._embed_documents(texts)
        
    @optimize_resource_usage(max_attempts=10)
    def _embed_query(self, text):
        return super()._embed_query(text)
        
    @optimize_resource_usage(max_attempts=10)
    def embed_documents(self, texts):
        return super().embed_documents(texts)
        
    @optimize_resource_usage(max_attempts=10)
    def embed_query(self, text):
        return super().embed_query(text)


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** These enhanced classes:

- Apply resource optimization to all critical API methods
- Maintain full compatibility with the standard LangChain interfaces
- Automatically handle intermittent service capacity issues
- Provide consistent behavior across both chat and embedding operations

## Building a Conversational Memory

One of the most important features of an effective chatbot is its ability to remember previous interactions. Without memory, each message would be treated in isolation, creating a disjointed experience.

In this section, we implement conversational memory using LangChain's `InMemoryChatMessageHistory` class. This allows our chatbot to reference previous exchanges and maintain context throughout the conversation.

### Conversation Formatting Utilities

This section implements a utility function that correctly formats multi-role conversations for large language models. The format follows the specific requirements of the Llama 3 model token format.

In [44]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## タスク 4.2: LangChain の chat history を使用して会話を開始する

このタスクでは、チャットボットがユーザーとの複数のやり取りにわたって会話のコンテキストを保持できるようにします。チャットボットが長期にわたって意味のある一貫した対話を行うには、会話メモリが不可欠です。

会話メモリ機能は、LangChain の InMemoryChatMessageHistory クラス上に構築して実装します。このオブジェクトには、ユーザーとチャットボット間の会話が保存され、チャットボットエージェントは履歴を利用できるため、以前の会話のコンテキストを活用できます。


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** モデルの出力は非決定的です。

In [45]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model = ResourceOptimizedChatBedrock(
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "以下の質問にできるだけ正確に答えてください。"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="お元気ですか？"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

Human: お元気ですか？
AI: はい、元気です。人工知能のアシスタントとして、体の健康状態を気にする必要はありませんが、適切に作動していることが何よりも大切です。このように、あなたと対話できることを嬉しく思っています。


### 新しい質問

モデルは最初のメッセージで応答しました。次に、モデルにいくつか質問します。

In [46]:
#new questions
instructions = [{"role": "user", "content": "新しくガーデニングを始めるためのヒントをいくつか教えてください。"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

始めてガーデニングをするなら、こんなヒントが役立つと思います。

1. 日当たりと水はけの良い場所を選ぶ
植物は十分な日光と適度な水分が必要です。日当たりと排水の良い場所を選びましょう。

2. 地域に適した植物を選ぶ
気候や土壌に合わない植物は育ちにくいので、地域に適した種類を選びましょう。園芸店や図書館で調べられます。

3. 良質の土を用意する
植物の健康な成長には良質の土が重要です。植木鉢用または庭園用の土を購入するのがおすすめです。

4. 水やりの量と頻度に気をつける
乾燥しすぎても湿りすぎても良くありません。土の表面を見ながら、適量と適度の頻度で水やりをしましょう。

5. 雑草対策をする
雑草は養分や水分を植物と奪い合うので、できるだけ早く取り除くことが大切です。

6. 少しずつ始める
ガーデニングは楽しい反面、手間もかかります。はじめは小規模から始め、徐々に広げていけば良いでしょう。

楽しみながら少しずつ経験を積めば、どんどん上手になっていくはずです。頑張ってください!


### 質問を基に構築

次に、ガーデニングという言葉を使わずに質問をして、モデルが前の会話を理解できるかどうかを確認します。

In [47]:
# build on the questions
instructions = [{"role": "user", "content": "虫についてはどうですか？"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

虫に関しても注意が必要ですね。

1. 害虫を認識する
アブラムシ、カメムシ、ハモグリバエなどの害虫を知り、早期に発見することが大切です。

2. 有機的な防除を試す
農薬を使わずに、石けん水やネームなどで防除を試みましょう。環境にも優しいですし、人体への影響も少ないでしょう。

3. 天敵を活用する
テントウムシやカブトムシなどの捕食性の昆虫は害虫の天敵です。有機農法で育てれば、天敵を呼び込むことができます。

4. 手で取り除く
少数の虫なら手で取り除くのも一つの方法です。虫が嫌いな方は手袋をするとよいでしょう。

5. 最悪の場合は農薬を使用する
有機的な方法でも防げない場合は、やむを得ず農薬を使用しますが、使用量や時期に気をつける必要があります。

虫との闘いはガーデニングの重要な側面です。観察を怠らず、環境に配慮しながら対処していきましょう。


### この会話を終える

In [48]:
# finishing the conversation
instructions = [{"role": "user", "content": "以上です、ありがとうございました！"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

私も質問に答えられて良かったです。ガーデニングは少し手間はかかりますが、植物の成長を見守り、実りを収穫できる喜びもあります。初めてでも、基本を押さえて少しずつ経験を積めば、きっと上手くいくはずです。虫との付き合い方も大切なポイントですね。楽しみながら、試行錯誤を重ねていけば自分なりのコツが見つかるはずです。ガーデニングを始められて良かったですね。頑張ってくださいね。


## タスク 4.3: プロンプトテンプレートを使用したチャットボット (LangChain)

### 一貫した対話のためのプロンプトテンプレートの使用

プロンプトエンジニアリングは、大規模言語モデルを扱う上で重要な側面です。テンプレートを使用することで、プロンプトを一貫した方法で構造化し、異なるシナリオでモデルがどのように応答するかを制御できます。

LangChain のプロンプトテンプレートは、異なる会話の役割（システム、ユーザー、アシスタント）に対する入力を標準化された方法でフォーマットし、モデルが適切に構造化されたコンテキストを受け取ることを保証します。

このタスクでは、この入力の構築を担当するデフォルトの PromptTemplate を使用します。LangChain は、プロンプトの構築と操作を容易にするためのクラスと関数をいくつか提供します。

In [49]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [50]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("ありがとう、楽しい会話でした!!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "回答なし"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q で終了します')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

次に、チャットを開始します。

In [51]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

Starting chat bot


Output()

Box(children=(Text(value='', description='You:', placeholder='q で終了します'), Button(description='Send', style=But…

In [ ]:
print(history)

## タスク 4.4: ペルソナ付きチャットボット

このタスクでは、人工知能 (AI) アシスタントがキャリアコー​​チの役割を果たします。システムメッセージを使用して、チャットボットにペルソナ (または役割) を通知できます。会話のコンテキストを維持するために、InMemoryChatMessageHistory クラスを引き続き活用します。

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたはキャリアコーチとして行動します。あなたの目標は、ユーザーにキャリアに関するアドバイスを提供することです。キャリアに関係のない質問に対しては、アドバイスを提供しないでください。「わかりません」と答えてください。"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "AI におけるキャリアの選択肢は何ですか?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "車を修理するにはどうすればいいですか?"})
print(response)

In [ ]:
print(history)

ここで、このペルソナの専門分野に属さない質問をします。モデルはその質問には答えず、その理由を説明する必要があります。

## タスク 4.5 コンテキスト付きチャットボット

このタスクでは、渡されたコンテキストに基づいてチャットボットに質問に答えるように依頼します。CSV ファイルを取得し、Titan 埋め込みモデルを使用してそのコンテキストを表すベクトルを作成します。このベクトルは Facebook AI 類似性検索 (FAISS) に保存されます。チャットボットに質問が出されたら、このベクトルをチャットボットに返し、ベクトルを使用して回答を取得させます。

### Titan 埋め込みモデル

埋め込みは、単語、フレーズ、またはその他の離散項目を連続ベクトル空間内のベクトルとして表します。これにより、機械学習モデルはこれらの表現に対して数学的演算を実行し、それらの間の意味関係をキャプチャできます。

埋め込みは、検索拡張生成 (RAG) [ドキュメント検索機能](https://labelbox.com/blog/how-vector-similarity-search-works/) に使用します。

In [ ]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = ResourceOptimizedEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=bedrock_client
)

### VectorStore としての FAISS

検索に埋め込みを使用するには、ベクトル類似性検索を効率的に実行できるストアが必要です。このノートブックでは、インメモリストアである FAISS を使用します。ベクトルを永続的に保存するには、Amazon Bedrock、pgVector、Pinecone、Weaviate、または Chroma の Knowledge Base を使用できます。

LangChain VectorStore API は [こちら](https://python.langchain.com/v0.2/docs/integrations/vectorstores/) から入手できます。

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("./Amazon_SageMaker_FAQs_jp.csv")
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

### 簡単なローコード テストを実行する

LangChain が提供する Wrapper クラスを使用して、ベクター データベース ストアをクエリし、関連するドキュメントを返すことができます。これにより、すべてのデフォルト値で QA チェーンが実行されます。

In [ ]:
chat_llm=ResourceOptimizedChatBedrock(
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("SageMakerにおけるR", llm=chat_llm))

### チャットボット アプリケーション

チャットボットには、コンテキスト管理、履歴、ベクターストア、その他多くのコンポーネントが必要です。まず、コンテキストをサポートする Retrieval Augmented Generation (RAG) チェーンを構築します。

これには、**create_stuff_documents_chain** 関数と **create_retrieval_chain** 関数が使用されます。

### RAG に使用されるパラメータ

- **Retriever:** `VectorStore` を基盤とする `VectorStoreRetriever` を使用しました。テキストを取得するには、`"similarity"` または `"mmr"` の 2 つの検索タイプから選択できます。`search_type="similarity"` は、リトリーバーオブジェクトで類似性検索を使用し、質問ベクトルに最も類似するテキストチャンクベクトルを選択します。

- **create_stuff_documents_chain** は、取得したコンテキストをプロンプトと LLM に取り込む方法を指定します。取得したドキュメントは、要約やその他の処理をプロンプトに施すことなく、コンテキストとして「詰め込まれ」ます。

- **create_retrieval_chain** は、取得ステップを追加し、取得したコンテキストをチェーンに伝播して、最終的な回答とともに提供します。

質問がコンテキストの範囲外である場合、モデルは答えがわからないと応答します。

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "あなたは質問回答タスクのアシスタントです。"
    "取得した下記のコンテキスト部分を使用して質問に回答します。 "
    "答えが分からない場合は、 「わかりません」と回答して下さい。 "
    "最大3文で簡潔に答えてください。 "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "SageMaker とは何ですか？"})
print(response) # shows the document chunks consulted to come up with the answer

次にチャットを開始します

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations

Claude LLM を利用して、次のパターンで会話型インターフェースを作成しました:

- チャットボット (基本 - コンテキストなし)
- プロンプトテンプレート (Langchain) を使用したチャットボット
- ペルソナ付きチャットボット
- コンテキスト付きチャットボット

### 試してみましょう

- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、**タスク 5** に進んでください。